# Import data

In [29]:
import pandas as pd

df_nonB = pd.read_csv('./data/genomes_nonB.csv')
df_nonB.shape

(103476, 6)

In [30]:
df_nonB.head()

,Unnamed: 0,names,subtypes,seqs,len_seqs,recomb
0,0,AY529677,C,atgagagtgatggggatacagaggaattgtcaacagtggtggatat...,2613,0
1,1,HQ595761,C,atgagagtgacggggatacggaagaattgtcaacaatggtggatat...,2535,0
2,2,HQ595746,C,atgagagtgatggggataacgaggaattgtcaacaatggtggatat...,2517,0
3,3,HM623585,C,atgagagtgagggggacatggaggaattatccacaatggtggatat...,2541,0
4,4,HM623566,C,atgagagtgaaggggatgcagaggaattgtccactatggtggatat...,2538,0


# Convert genomes to trigrams

In [31]:
sequences = df_nonB['seqs'].values

In [32]:
from nltk import ngrams
import numpy as np

corpus = []
for seq in sequences:
    in_tri = np.array([''.join(i) for i in ngrams(seq, 3)])
    corpus.append(in_tri)

In [33]:
sentences = [' '.join(x) for x in corpus]

y = df_nonB['recomb'].values

# Train test split

In [34]:
from sklearn.model_selection import train_test_split

sentences_train, sentences_test, y_train, y_test = train_test_split(sentences,
                                                y, test_size=0.25,
                                                random_state=100, stratify=y)

# Tokenize the ngrams

In [35]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(sentences_train)

X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

# Padding to max 3000

In [36]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 3000

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

print(X_train[0, :])

[54 23 40 ...  0  0  0]


# Models

In [37]:
import keras

METRICS = [
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.AUC(name='auc')]

In [38]:
import tensorflow as tf

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc', 
    verbose=1,
    patience=5,
    mode='max',
    restore_best_weights=True)

## Baseline model

In [39]:
from keras.models import Sequential
from keras import layers

embedding_dim = 50

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.Flatten())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=[METRICS])

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 3000, 50)          83250     
_________________________________________________________________
flatten_3 (Flatten)          (None, 150000)            0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                1500010   
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 11        
Total params: 1,583,271
Trainable params: 1,583,271
Non-trainable params: 0
_________________________________________________________________


In [40]:
history = model.fit(X_train, y_train,
                    epochs=30,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    callbacks = [early_stopping],
                    batch_size=256)

loss, accuracy, auc = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
print("Training AUC: {:.4f}".format(auc))
loss, accuracy, auc = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
print("Testing AUC:  {:.4f}".format(auc))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 77607 samples, validate on 25869 samples
Epoch 1/10
77607/77607 [==============================] - 13s 165us/step - loss: 0.1445 - accuracy: 0.9389 - auc: 0.9872 - val_loss: 0.0681 - val_accuracy: 0.9748 - val_auc: 0.9966
Epoch 2/10
77607/77607 [==============================] - 12s 159us/step - loss: 0.0439 - accuracy: 0.9852 - auc: 0.9982 - val_loss: 0.0689 - val_accuracy: 0.9759 - val_auc: 0.9960
Epoch 3/10
77607/77607 [==============================] - 12s 159us/step - loss: 0.0302 - accuracy: 0.9910 - auc: 0.9989 - val_loss: 0.0617 - val_accuracy: 0.9795 - val_auc: 0.9968
Epoch 4/10
77607/77607 [==============================] - 12s 159us/step - loss: 0.0221 - accuracy: 0.9937 - auc: 0.9993 - val_loss: 0.0625 - val_accuracy: 0.9795 - val_auc: 0.9963
Epoch 5/10
77607/77607 [==============================] - 12s 159us/step - loss: 0.0173 - accuracy: 0.9945 - auc: 0.9996 - val_loss: 0.0718 - val_accuracy: 0.9781 - val_auc: 0.9959
Epoch 6/10
77607/77607 [=====================

## Add a conv1d

In [47]:
embedding_dim = 50

METRICS = [
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.AUC(name='auc')]



model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 3000, 50)          83250     
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 2996, 128)         32128     
_________________________________________________________________
global_max_pooling1d_8 (Glob (None, 128)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 10)                1290      
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 11        
Total params: 116,679
Trainable params: 116,679
Non-trainable params: 0
_________________________________________________________________


In [48]:
history = model.fit(X_train, y_train,
                    epochs=30,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    callbacks = [early_stopping],
                    batch_size=256)

loss, accuracy, auc = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
print("Training AUC: {:.4f}".format(auc))
loss, accuracy, auc = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
print("Testing AUC:  {:.4f}".format(auc))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 77607 samples, validate on 25869 samples
Epoch 1/30
77607/77607 [==============================] - 62s 797us/step - loss: 0.2165 - accuracy: 0.9024 - auc: 0.9759 - val_loss: 0.0683 - val_accuracy: 0.9782 - val_auc: 0.9970
Epoch 2/30
77607/77607 [==============================] - 61s 785us/step - loss: 0.0570 - accuracy: 0.9807 - auc: 0.9976 - val_loss: 0.0486 - val_accuracy: 0.9846 - val_auc: 0.9980
Epoch 3/30
77607/77607 [==============================] - 61s 786us/step - loss: 0.0440 - accuracy: 0.9854 - auc: 0.9984 - val_loss: 0.0447 - val_accuracy: 0.9860 - val_auc: 0.9984
Epoch 4/30
77607/77607 [==============================] - 61s 786us/step - loss: 0.0377 - accuracy: 0.9874 - auc: 0.9987 - val_loss: 0.0468 - val_accuracy: 0.9853 - val_auc: 0.9983
Epoch 5/30
77607/77607 [==============================] - 61s 785us/step - loss: 0.0340 - accuracy: 0.9888 - auc: 0.9989 - val_loss: 0.0383 - val_accuracy: 0.9879 - val_auc: 0.9985
Epoch 6/30
77607/77607 [=====================

In [49]:
from sklearn.metrics import classification_report

y_pred = model.predict_classes(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     15034
           1       0.99      0.99      0.99     10835

    accuracy                           0.99     25869
   macro avg       0.99      0.99      0.99     25869
weighted avg       0.99      0.99      0.99     25869

